In [1]:
%%time
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE,RandomOverSampler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from itertools import combinations 
from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
import copy 

Using TensorFlow backend.
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sf/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtype

CPU times: user 2.42 s, sys: 545 ms, total: 2.97 s
Wall time: 2.62 s


In [2]:
df=pd.read_csv('60s_window_wrist_chest.csv',index_col=0)


In [3]:
df=df[df['label']<2]
df['label'].value_counts()


0    312
1    175
Name: label, dtype: int64

In [4]:
features=df.columns.tolist()
features

removed = ['label']
for rem in removed:
    features.remove(rem)

features_with_sub=[]
features_with_sub[:]=features
removed = ['subject']
for rem in removed:
    features.remove(rem)

feature=features
print(len(feature))
len(features_with_sub)


70


71

In [5]:
sm = SMOTE(random_state=2)
X, y= sm.fit_sample(df[features_with_sub], df['label'])
df_new=pd.concat([pd.DataFrame(X,columns=features_with_sub),pd.DataFrame(y,columns=['label'])],axis=1)
df_new

,net_acc_mean,net_acc_std,net_acc_min,net_acc_max,ACC_x_mean,ACC_x_std,ACC_x_min,ACC_x_max,ACC_y_mean,ACC_y_std,...,c_ACC_z_min,c_ACC_z_max,c_Temp_mean,c_Temp_std,c_Temp_min,c_Temp_max,BVP_peak_freq,TEMP_slope,subject,label
0,0.025961,0.013811,0.000000,0.087383,0.023431,0.017769,-0.037843,0.087383,0.000016,1.222594e-05,...,-0.870000,0.611000,29.168923,0.064290,28.994568,29.426208,0.081425,-0.000253,2.000000,0
1,0.027640,0.010597,0.002752,0.054356,0.027640,0.010597,0.002752,0.054356,0.000019,7.290999e-06,...,-0.759400,-0.681000,28.886605,0.074846,28.730682,29.207275,0.147017,-0.000161,2.000000,0
2,0.028389,0.006937,0.000000,0.066053,0.028378,0.006985,-0.002752,0.066053,0.000020,4.805734e-06,...,-0.753400,-0.675400,28.799659,0.037924,28.679108,28.988800,0.088210,0.000535,2.000000,0
3,0.033268,0.007670,0.000000,0.074998,0.032960,0.008904,-0.030962,0.074998,0.000023,6.126303e-06,...,-0.787800,0.166000,28.768865,0.058639,28.584656,29.023285,0.117614,-0.000256,2.000000,0
4,0.037021,0.001284,0.027522,0.043347,0.037021,0.001284,0.027522,0.043347,0.000025,8.837530e-07,...,-0.720200,-0.657000,28.598514,0.068128,28.447449,28.882599,0.151541,0.000260,2.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,0.028499,0.005744,0.009549,0.049915,-0.006100,0.005744,-0.015845,0.024521,-0.000004,3.952020e-06,...,-0.540288,0.048413,34.173330,0.037962,34.020692,34.347792,0.150990,-0.000174,15.151558,1
620,0.037179,0.001151,0.023449,0.050451,-0.001681,0.001151,-0.014134,0.012868,-0.000001,7.917207e-07,...,-0.749950,-0.539231,34.179298,0.031306,34.074914,34.339609,0.123940,0.000388,12.392020,1
621,0.037896,0.004781,0.003547,0.056589,0.037896,0.004781,0.003547,0.056589,0.000026,3.289295e-06,...,-0.374418,0.053229,34.265974,0.034625,34.133383,34.413883,0.085883,0.000197,15.314008,1
622,0.028627,0.002078,0.019131,0.044707,0.021569,0.002078,0.012419,0.037995,0.000015,1.429441e-06,...,-0.328649,0.046121,34.319910,0.027538,34.228198,34.463565,0.122004,-0.000229,15.512260,1


In [6]:
for i in range (len(list(df_new['subject']))):
    df_new['subject'][i] = min([2,3,4,5,6,7,8,9,10,11,13,14,15,16,17], key=lambda x:abs(x-df_new['subject'][i]))
df_new['subject']=df_new['subject'].astype(int)

/home/sf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
p_d=pd.read_csv('personal_detail.csv',index_col=0)

In [8]:
df_new_1=df_new.merge(p_d,on='subject')
df_new_1

,net_acc_mean,net_acc_std,net_acc_min,net_acc_max,ACC_x_mean,ACC_x_std,ACC_x_min,ACC_x_max,ACC_y_mean,ACC_y_std,...,label,age,height,weight,gender_ female,coffee_today_YES,sport_today_YES,smoker_YES,feel_ill_today_YES,bmi
0,0.025961,0.013811,0.000000,0.087383,0.023431,0.017769,-0.037843,0.087383,0.000016,1.222594e-05,...,0,27,175,80,0,0,0,0,0,1
1,0.027640,0.010597,0.002752,0.054356,0.027640,0.010597,0.002752,0.054356,0.000019,7.290999e-06,...,0,27,175,80,0,0,0,0,0,1
2,0.028389,0.006937,0.000000,0.066053,0.028378,0.006985,-0.002752,0.066053,0.000020,4.805734e-06,...,0,27,175,80,0,0,0,0,0,1
3,0.033268,0.007670,0.000000,0.074998,0.032960,0.008904,-0.030962,0.074998,0.000023,6.126303e-06,...,0,27,175,80,0,0,0,0,0,1
4,0.037021,0.001284,0.027522,0.043347,0.037021,0.001284,0.027522,0.043347,0.000025,8.837530e-07,...,0,27,175,80,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,0.031629,0.001143,0.028210,0.039219,-0.031629,0.001143,-0.039219,-0.028210,-0.000022,7.864926e-07,...,1,29,165,55,1,0,0,0,0,0
620,0.033437,0.001540,0.013073,0.051604,-0.033437,0.001540,-0.051604,-0.013073,-0.000023,1.059291e-06,...,1,29,165,55,1,0,0,0,0,0
621,0.031757,0.001600,0.013761,0.037155,-0.031757,0.001600,-0.037155,-0.013761,-0.000022,1.101023e-06,...,1,29,165,55,1,0,0,0,0,0
622,0.029441,0.002093,0.020770,0.054100,-0.004075,0.002093,-0.014956,0.018375,-0.000003,1.440095e-06,...,1,29,165,55,1,0,0,0,0,0


In [9]:
df_new_1['label'].value_counts()

1    312
0    312
Name: label, dtype: int64

In [10]:
features=df_new_1.columns.tolist()
features

removed = ['label']
for rem in removed:
    features.remove(rem)
features_with_sub=[]
features_with_sub[:]=features
removed = ['subject']
for rem in removed:
    features.remove(rem)

feature=features
print(len(feature))
len(features_with_sub)

79


80

In [11]:
train=df_new_1[df_new_1['subject']<=9]
test=df_new_1[df_new_1['subject']>9]

In [12]:
# scaler = Normalizer()
# scaled_data_train = scaler.fit_transform(train[feature])
# scaled_data_test = scaler.transform(test[feature])

In [13]:
et = ExtraTreesClassifier(n_estimators=50,n_jobs=10,random_state=70)
et.fit(train[feature],train['label'])
y_pred=et.predict(test[feature])

In [14]:
print(classification_report(test['label'],y_pred))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       147
           1       0.96      0.98      0.97       161

    accuracy                           0.96       308
   macro avg       0.96      0.96      0.96       308
weighted avg       0.96      0.96      0.96       308



In [15]:
feature_importances = pd.DataFrame(et.feature_importances_,index = feature,columns=['importance']).sort_values('importance', ascending=False)

In [16]:
feature_importances

,importance
EDA_tonic_mean,0.113696
EDA_tonic_max,0.079433
EDA_tonic_min,0.077148
EDA_smna_std,0.055666
Resp_std,0.054057
...,...
coffee_today_YES,0.000957
c_Temp_std,0.000870
ECG_mean,0.000817
EDA_smna_min,0.000536


# Feature Selection with feature_importances using forward Selection


In [17]:
feature_importances.index

Index(['EDA_tonic_mean', 'EDA_tonic_max', 'EDA_tonic_min', 'EDA_smna_std',
       'Resp_std', 'Resp_max', 'EDA_phasic_max', 'TEMP_mean', 'EDA_smna_max',
       'EDA_max', 'EDA_min', 'TEMP_min', 'c_ACC_z_mean', 'EDA_mean',
       'EDA_smna_mean', 'EDA_phasic_min', 'EDA_phasic_mean', 'c_ACC_x_max',
       'Resp_min', 'c_ACC_x_mean', 'c_ACC_z_max', 'TEMP_slope', 'ECG_max',
       'TEMP_max', 'net_acc_mean', 'EDA_phasic_std', 'EDA_tonic_std',
       'c_ACC_x_std', 'c_Temp_mean', 'ACC_x_mean', 'ECG_min', 'ACC_y_mean',
       'EMG_mean', 'BVP_peak_freq', 'ACC_z_max', 'c_ACC_y_max', 'c_Temp_max',
       'height', 'TEMP_std', 'c_ACC_z_min', 'ACC_y_max', 'ACC_x_max',
       'ECG_std', 'EMG_max', 'sport_today_YES', 'ACC_z_mean', 'EDA_std',
       'c_ACC_x_min', 'BVP_min', 'EMG_std', 'ACC_z_min', 'ACC_x_std',
       'c_ACC_y_mean', 'ACC_z_std', 'EMG_min', 'feel_ill_today_YES',
       'net_acc_std', 'c_ACC_z_std', 'bmi', 'net_acc_max', 'ACC_y_std',
       'BVP_std', 'c_ACC_y_min', 'weight', 'age',

In [18]:
# sel_fea = ['EDA_tonic_mean','EDA_tonic_max','EDA_phasic_max','TEMP_mean','EDA_max']
sel_fea = []
pas_acc = 0.0

for i in feature_importances.index:
    sel_fea.append(i)
    print (len(sel_fea))
    et = ExtraTreesClassifier(n_estimators=50,n_jobs=10,random_state=70)
    et.fit(train[sel_fea],train['label'])
    y_pred=et.predict(test[sel_fea])

    rpt = classification_report(test['label'],y_pred,output_dict=True)['accuracy']
    
    if rpt< pas_acc:
        sel_fea.pop()
    else:
        pas_acc = rpt

1
2
3
4
5
5
6
7
8
9
9
9
10
10
10
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14


In [19]:
sel_fea

['EDA_tonic_mean',
 'EDA_tonic_max',
 'EDA_tonic_min',
 'EDA_smna_std',
 'Resp_max',
 'EDA_phasic_max',
 'TEMP_mean',
 'EDA_smna_max',
 'TEMP_min',
 'EDA_smna_mean',
 'EDA_phasic_mean',
 'ECG_std',
 'EMG_max']

In [34]:
sel_fea_tuned = ['TEMP_min','EDA_tonic_max','EDA_phasic_max','ECG_std']

et = ExtraTreesClassifier(n_estimators=50,n_jobs=10,random_state=70)
et.fit(train[sel_fea_tuned],train['label'])
y_pred=et.predict(test[sel_fea_tuned])
    

rpt = classification_report(test['label'],y_pred,output_dict=True)['accuracy']
print (rpt)

0.9448051948051948


In [ ]:
for i in range (0,101):
    et = ExtraTreesClassifier(n_estimators=50,n_jobs=10,random_state=i)
    et.fit(train[sel_fea_tuned],train['label'])
    y_pred=et.predict(test[sel_fea_tuned])

    rpt = classification_report(test['label'],y_pred,output_dict=True)['accuracy']
    
    if rpt >= 0.95:
        print (i)
        print (classification_report(test['label'],y_pred))
